In [ ]:
#| include: false

import os
import pandas as pd
import numpy as np
# import plotly.express as px
import toml
# import psrc_theme
from pathlib import Path
import summary_data

# to show plotly figures in quarto HTML file
# import plotly.io as pio
# pio.renderers.default = "plotly_mimetype+notebook_connected"
# pio.templates.default = "simple_white+psrc_color" # set plotly template

config = summary_data.CONFIG
all_runs = summary_data.ALL_RUNS

In [75]:
# network summary: 'VMT','VHT','total_delay'
df_network = summary_data.load_agg_data('network/network_results.csv')
network_summary = df_network.groupby('source')[['VMT','VHT','total_delay']].sum()
network_summary = network_summary.rename(columns={'total_delay': 'Delay'})
# transit boardings
df_boardings = summary_data.load_agg_data('transit/daily_boardings_by_agency.csv')
transit_ridership = df_boardings.groupby('source')['boardings'].sum()
# mode share
df_trip = summary_data.load_agg_data('agg/dash/mode_share_county.csv')
df = df_trip.groupby('source')['trexpfac'].sum() # total trips
transit_share = df_trip.loc[df_trip['mode']=="Transit"].groupby('source')['trexpfac'].sum()/df

# emission
df_emissions = summary_data.load_agg_data('emissions/emissions_summary.csv')
df = df_emissions.loc[(df_emissions['veh_type'].isin(['light','medium','heavy'])) & \
                      (df_emissions['pollutant_name']=="CO2 Equivalent")].copy()
CO2e = df.groupby('source')['total_daily_tons'].sum()

In [76]:
run_summary2 = pd.DataFrame({
    'Transit Boardings': transit_ridership,
    '% Transit': transit_share,
    'CO2e': CO2e
})

pd.concat([network_summary,run_summary2], axis=1).style.\
        format('{:,.0f}', subset=['VMT','VHT','Delay','Transit Boardings','CO2e']).\
        format('{:.1%}', subset=['% Transit'])

,VMT,VHT,Delay,Transit Boardings,% Transit,CO2e
source,,,,,,
current run,"74,794,942","2,216,555","195,921","480,586",2.0%,"41,543"
i5 toll,"105,206,842","3,511,647","595,756","2,331,178",6.2%,"37,407"


In [95]:
df_network['data3'].value_counts()

3.0    1009321
5.0     433536
6.0     179530
2.0      76133
1.0      66442
4.0      58022
8.0        504
Name: data3, dtype: int64

## population & households

In [67]:
def count_by_county(output_path: str, summary_var: str, expfac_name: str, axis_name: str):
    df = summary_data.load_agg_data(output_path)

    tab = df.groupby(['source',summary_var])[expfac_name].sum().\
        unstack(summary_var).\
        rename_axis(columns={summary_var: axis_name})
    tab['Total'] = tab.sum(axis=1)

    display(tab.style.format('{:,.0f}'))

In [72]:
count_by_county('agg/dash/pptyp_county.csv','person_county','psexpfac','Population by county')

Population by county,King,Kitsap,Pierce,Snohomish,Total
source,,,,,
current run,"2,303,921","275,126","925,392","851,593","4,356,032"
i5 toll,"3,029,767","353,461","1,213,634","1,230,341","5,827,203"


In [73]:
count_by_county('agg/dash/hh_geog.csv','hh_county','hhexpfac','Households by county')

Households by county,King,Kitsap,Pierce,Snohomish,Total
source,,,,,
current run,"957,477","108,903","350,089","319,660","1,736,129"
i5 toll,"1,310,283","147,388","487,287","477,645","2,422,603"


## employment by work location

In [74]:
count_by_county('agg/dash/person_worker_type.csv','person_work_county','psexpfac','Employment by county')

Employment by county,King,Kitsap,Pierce,Snohomish,Total
source,,,,,
current run,"1,285,062","83,946","285,435","250,288","1,904,731"
i5 toll,"1,731,184","132,330","373,101","403,608","2,640,223"


## vehicle miles traveled

In [88]:

df = df_network.groupby(['source','county'])['VMT'].sum().\
        unstack('county').\
        rename_axis(columns={'county': 'VMT by county'})
tab = df[['King','Kitsap','Pierce','Snohomish','Outside Region']].copy()
tab['Total'] = tab.sum(axis=1)

display(tab.style.format('{:,.0f}'))

VMT by county,King,Kitsap,Pierce,Snohomish,Outside Region,Total
source,,,,,,
current run,"39,769,936","3,910,219","16,573,613","14,235,525","305,650","74,794,942"
i5 toll,"53,904,151","5,997,860","22,939,637","21,852,307","512,887","105,206,842"


In [89]:
df_network['facility_type'].value_counts()

Arterial     1246873
Connector     433536
Highway       142575
Other            504
Name: facility_type, dtype: int64

In [94]:
df = df_network.groupby(['source','facility_type'])['VMT'].sum().\
        unstack('facility_type').\
        rename_axis(columns={'facility_type': 'VMT by facility type'}).\
        drop(columns='Other')
df['Total'] = df.sum(axis=1)

display(df.style.format('{:,.0f}'))

VMT by facility type,Arterial,Connector,Highway,Total
source,,,,
current run,"31,726,501","5,275,254","37,784,698","74,786,453"
i5 toll,"48,491,547","6,912,928","49,802,367","105,206,842"


In [ ]:
df_tod = df.groupby(['period']).sum()[['arterial','connector','highway']]
df_tod['VMT'] = df_tod.sum(axis=1)
df_tod.loc['Total'] = df_tod.sum()
df_tod = df_tod[['VMT']]

# Save total VMT for later use
total_vmt = df_tod.loc['Total'].values[0]

df_fac = pd.DataFrame(df.sum()[['arterial','highway','connector']])
df_fac = df_fac.reset_index()
df_fac.columns = ['Facility Type', 'VMT']
df_fac.index = df_fac['Facility Type']
df_fac.drop('Facility Type', axis=1, inplace=True)
df_fac.loc['Total'] = df_fac.sum()

df_fac.to_csv(os.path.join(os.getcwd(), r'..\..\..\outputs\compare\topsheet','1_vmt_facility_type.csv'))
df_tod.to_csv(os.path.join(os.getcwd(), r'..\..\..\outputs\compare\topsheet','2_vmt_tod.csv'))
display(df_fac)
display(df_tod)